In [1]:
import bitstruct as bs
import numpy as np
import pandas as pd

In [2]:
file = 'Export_INT_GC1_MC2_RC3_S3_Full_AOI_DTID_GUANN_DEBUG.xlsx'

In [3]:
df = pd.read_excel(file, engine='openpyxl')

In [4]:
df.head()

,Unnamed: 0,Start,End,Duration,Satellite,SensorMode,Width,Length,SwathArea,Pass,...,Ground_User_Annotation,TAnx_integer,Phase,Drift,GCID,MCID,RCID,Track,ExtCal,Spare
0,0,2017-03-02 06:07:23.674,2017-03-02 06:09:37.671,133.997563,BIOMASS,Stripmap S3,45.726444,920.782773,42272.374464,ASCENDING,...,11000010010001100000001000000000,92,3,0,1,2,3,1,0,0
1,1,2017-03-02 06:58:33.629,2017-03-02 06:59:05.629,32.000000,BIOMASS,Stripmap S3,46.066410,218.881989,10115.600647,DESCENDING,...,11000010010001100000001000000000,3162,3,0,1,2,3,1,0,0
2,2,2017-03-02 07:00:13.784,2017-03-02 07:00:45.784,32.000000,BIOMASS,Stripmap S3,45.972871,218.602453,10129.389672,DESCENDING,...,11000010010001100000001000000000,3262,3,0,1,2,3,1,0,0
3,3,2017-03-02 07:12:37.221,2017-03-02 07:26:17.822,820.601003,BIOMASS,Stripmap S3,47.620119,5547.133054,263829.938896,DESCENDING,...,11000010010001100000001000000000,4005,3,0,1,2,3,1,0,0
4,4,2017-03-02 08:33:54.289,2017-03-02 08:42:30.585,516.295486,BIOMASS,Stripmap S3,46.166360,3527.657239,163411.947057,DESCENDING,...,11000010010001100000010000000000,2991,3,0,1,2,3,2,0,0


Check `TAnx_integer`

In [5]:
(df['TAnx'].astype(int) == df['TAnx_integer']).all()

True

Check `DataTake_ID`

In [6]:
dt_id = df.apply(lambda s: int.from_bytes(bs.pack('>u19u13', s['Orbit'], s['TAnx_integer']), 'big'), axis=1)

dt_id

0      7217244
1      7220314
2      7220414
3      7221157
4      7228335
        ...   
158    7561241
159    7561833
160    7565255
161    7566565
162    7573447
Length: 163, dtype: int64

In [7]:
(dt_id.apply(lambda d: format(d, '032b')) == df['DataTake_ID']).all()

True

Check `MCID`

In [8]:
swaths = 3
rc = 44
orbits_per_mc = np.ceil(rc * 3 * swaths + rc * 3)

In [9]:
(np.ceil(df['Orbit'] / orbits_per_mc).astype(int) == df['MCID']).all()

True

In [10]:
df['MCID'].unique()

array([2])

Check `RCID`

In [11]:
orbits_per_swath = rc * 3

rcid = np.ceil(df['Orbit'] % orbits_per_mc / orbits_per_swath).astype(int)

(rcid == df['RCID']).all()

True

In [12]:
rcid.unique()

array([3])

Check `Track`

In [13]:
track = df['Orbit'].apply(lambda o: o % rc or rc)

(track == df['Track']).all()

True

In [14]:
track

0       1
1       1
2       1
3       1
4       2
       ..
158    43
159    43
160    43
161    43
162    44
Name: Orbit, Length: 163, dtype: int64

In [15]:
phase = 3
drift = 0
gcid = 1
extcal = 0

In [16]:
guann = df.apply(lambda s: int.from_bytes(bs.pack('>u2u1u4u4u4u8u2u7', phase, drift, gcid, s['MCID'], s['RCID'], s['Track'], extcal, 0), 'big'), axis=1)

In [17]:
(guann.apply(lambda s: format(s, '032b')) == df['Ground_User_Annotation']).all()

True